In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
BATCH_SIZE = 128
SEQ_LEN = 256
EPOCHS = 4000
lr = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMBEDDING_DIM = 384
NUM_HEADS = 6
NUM_BLOCKS = 6
DROPOUT_RATE = 0.4

torch.manual_seed(69)

In [4]:
with open('/content/drive/MyDrive/nlp/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
# create a mapping from characters to integers
CHAR_TO_INDEX = { ch:i for i,ch in enumerate(chars) }
INDEX_TO_CHAR = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [CHAR_TO_INDEX[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([INDEX_TO_CHAR[i] for i in l]) # decoder: take a list of integers, output a string

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
pct = int(0.9*len(data))
train_data = data[:pct]
val_data = data[pct:]

In [7]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - SEQ_LEN, (BATCH_SIZE,))
    x = torch.stack([data[i:i+SEQ_LEN] for i in ix])
    y = torch.stack([data[i+1:i+SEQ_LEN+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [8]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(EMBEDDING_DIM, 4*EMBEDDING_DIM)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4*EMBEDDING_DIM, EMBEDDING_DIM)
        self.dropout = nn.Dropout(DROPOUT_RATE)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

In [9]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.query = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.value = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.register_buffer('tril', torch.tril(torch.ones(SEQ_LEN, SEQ_LEN)))

    def forward(self, x):
        B,S,C = x.shape
        k = self.key(x)   # (B,S,C)
        q = self.query(x) # (B,S,C)
        v = self.value(x) # (B,S,C)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, S, C) @ (B, C, S) -> (B, S, S)
        wei = wei.masked_fill(self.tril[:S, :S] == 0, float('-inf')) # (B, S, S)
        wei = F.softmax(wei, dim=-1) # (B, S, S)
        wei = self.dropout(wei)
        out = wei @ v # (B, S, S) @ (B, S, C) -> (B, S, C)
        return out

In [10]:
class MultipleHead(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.fc = nn.Linear(num_heads*head_size, EMBEDDING_DIM)
        self.dropout = nn.Dropout(DROPOUT_RATE)

    def forward(self, x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.fc(x)
        x = self.dropout(x)
        return x

In [11]:
class Block(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        head_size = EMBEDDING_DIM // num_heads
        self.ln1 = nn.LayerNorm(EMBEDDING_DIM)
        self.ln2 = nn.LayerNorm(EMBEDDING_DIM)
        self.ff = FeedForward()
        self.mh = MultipleHead(num_heads, head_size)

    def forward(self, x):
        x = x + self.mh(x)
        x = self.ln1(x)
        x = x + self.ff(x)
        x = self.ln2(x)
        return x

In [12]:
class Attention(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.position_embedding_table = nn.Embedding(SEQ_LEN, EMBEDDING_DIM)
        self.blocks = nn.ModuleList([Block(NUM_HEADS) for _ in range(NUM_BLOCKS)])
        self.fc = nn.Linear(EMBEDDING_DIM, VOCAB_SIZE)
        self.ln = nn.LayerNorm(EMBEDDING_DIM)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        for block in self.blocks:
            x = block(x)
        x = self.ln(x)
        logits = self.fc(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -SEQ_LEN:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [13]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=lr)
model = model.to(DEVICE)

In [14]:
model.to(DEVICE)
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}", end=' ')
    model.train()
    x, y = get_batch("train")
    optimizer.zero_grad()
    y_pred, loss = model(x, y)
    print(f"Training Loss: {loss} -", end=' ')
    loss.backward()
    optimizer.step()
    model.eval()
    x, y = get_batch("val")
    y_pred, loss = model(x, y)
    print(f"Validation Loss: {loss}")

Training Loss: 2.7185394763946533 - Validation Loss: 2.6767334938049316
Epoch 48 Training Loss: 2.7045183181762695 - Validation Loss: 2.6821930408477783
Epoch 49 Training Loss: 2.6965296268463135 - Validation Loss: 2.6708004474639893
Epoch 50 Training Loss: 2.6963260173797607 - Validation Loss: 2.6909639835357666
Epoch 51 Training Loss: 2.6995818614959717 - Validation Loss: 2.671504497528076
Epoch 52 Training Loss: 2.6818509101867676 - Validation Loss: 2.655852794647217
Epoch 53 Training Loss: 2.6701626777648926 - Validation Loss: 2.6333868503570557
Epoch 54 Training Loss: 2.6782472133636475 - Validation Loss: 2.647613525390625
Epoch 55 Training Loss: 2.6578168869018555 - Validation Loss: 2.6663286685943604
Epoch 56 Training Loss: 2.6582655906677246 - Validation Loss: 2.6369946002960205
Epoch 57 Training Loss: 2.6453518867492676 - Validation Loss: 2.618328094482422
Epoch 58 Training Loss: 2.6441848278045654 - Validation Loss: 2.6353061199188232
Epoch 59 Training Loss: 2.655592441558838

In [18]:
# context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
context = 'JULIET'
context = torch.tensor(encode(context), dtype=torch.long, device=DEVICE).unsqueeze(0)
context = context.to(DEVICE)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))

JULIET:
You shall you any depty man tempt the arm.
But, which that great is Warwick's your counting?

DUCHESS OF YORK:
By good lords, thou must than my comfort.

KING RICHARD II:
So did not for the rest delives
Whom we strants up hight free to be one.

KING RICHARD II:
O, better below so lawger them looks: must I have all
Upon thee one. God could my lord, any gentleman
Do heart upon my lord honour's did not cher here: I wrett thee,
If they prince, gave defully have safect-royal
I be son the worth unleasure's chancers, bear itself:
He's booth-deside to the deed of peope their unjudesty
Speak.

GLOUCESTER:
The sath! what, I prince not not man! Why that?

LUCIO:
By all is thou the good love thee to you here?

MENENIUS:

MENENIUS:
What's thou?

VOLUMNIA:
Now; one my lord?

Second Murder:
But what have speak he sone is this birning,
'The which I take be any all gentle.

AUTOLYCUS:
Nay, and my general: you yet the husband by that hath
With been his rather, as you love at old:
What voice, my 

In [19]:
torch.save(model.state_dict(), '/content/drive/MyDrive/nlp/model.pth')